In [47]:
import pandas as pd
import numpy as np
import re

## v2.0 - sca open data

In [30]:
gh = pd.read_csv('https://github.com/NYCPlanning/db-scacapitalprojects/raw/master/scacapitalprojects_build/output/sca_cp_capacity_projects.csv')

In [53]:
def generate_datetime(x):
    ar = x.split('-')
    try:
        int(ar[0])
        return pd.to_datetime(x, format="%y-%b")
    except ValueError:
        return pd.to_datetime(x, format="%b-%y")

In [57]:
gh = gh[gh.description == 'capacity projects']

In [59]:
gh['designstart'] = gh.designstart.apply(generate_datetime)

In [60]:
gh['constrstart'] = gh.constrstart.apply(generate_datetime)
gh['actualestcompl'] = gh.actualestcompl.apply(generate_datetime)

In [66]:
locations.dtypes

School Name               object
BoroughCode               object
Geographical District      int64
Project Description       object
Construction Award       float64
Project type              object
Building ID               object
Building Address          object
City                      object
Postcode                 float64
Borough                   object
Latitude                 float64
Longitude                float64
Community Board          float64
Community Council        float64
Census Tract             float64
BIN                      float64
BBL                      float64
NTA                       object
Location 1                object
Data As Of                object
dtype: object

## v1.0 - merging projects

In [2]:
# Data from: https://data.cityofnewyork.us/Housing-Development/Capital-Project-Schedules-and-Budgets/2xh6-psuq
# As of: 02/09/2018

budgets = pd.read_csv('data/Capital_Project_Schedules_and_Budgets.csv')

In [3]:
budgets.sample()

,Project Geographic District,Project Building Identifier,Project School Name,Project Type,Project Description,Project Phase Name,Project Status Name,Project Phase Actual Start Date,Project Phase Planned End Date,Project Phase Actual End Date,Project Budget Amount,Final Estimate of Actual Costs Through End of Phase Amount,Total Phase Actual Spending Amount,DSF Number(s)
5731,22,K193,P.S. 193 - BROOKLYN,DIIT - Project Conne,CLASSROOM CONNECTIVITY,Purch & Install,In-Progress,06/24/2013,12/31/2013,NaN,DIIT,$245683.00,$0.00,DSF: 0000776802


In [4]:
budgets = budgets[(budgets['Project Type '] == 'SCA Capacity') & (budgets['Project Status Name'] == 'In-Progress')]

In [26]:
budgets.sample(5)

,Project Geographic District,Project Building Identifier,Project School Name,Project Type,Project Description,Project Phase Name,Project Status Name,Project Phase Actual Start Date,Project Phase Planned End Date,Project Phase Actual End Date,Project Budget Amount,Final Estimate of Actual Costs Through End of Phase Amount,Total Phase Actual Spending Amount,DSF Number(s)
6696,24,Q019,P.S. 19 - QUEENS,SCA Capacity,ADDITION,Construction,In-Progress,07/18/2016,06/17/2019,NaN,107141924,$97171330.00,$42139303.00,DSF: 0000798568
4938,20,K746,P.S. 746 - BROOKLYN,SCA Capacity,NEW SCHOOL,"CM,Art,F&E",In-Progress,06/22/2017,03/07/2020,NaN,14654026,$5073355.00,$401868.00,DSF: 0000798186
4540,25,Q024,P.S. 24 - QUEENS,SCA Capacity,ADDITION,Construction,In-Progress,07/26/2016,09/18/2019,NaN,92510022,$84913231.00,$21915106.00,DSF: 0000798203
6704,24,Q049,P.S. 49 - QUEENS,SCA Capacity,ADDITION,Construction,In-Progress,12/22/2015,12/18/2017,NaN,35381148,$33096938.00,$31499499.00,DSF: 0000798200
7952,31,R462,SUSAN E. WAGNER HS,SCA Capacity,ADDITION,Construction,In-Progress,09/22/2014,07/12/2016,NaN,28029692,$27132495.00,$23910527.00,DSF: 0000775719


In [25]:
budgets.dtypes

Project Geographic District                                    int64
Project Building Identifier                                   object
Project School Name                                           object
Project Type                                                  object
Project Description                                           object
Project Phase Name                                            object
Project Status Name                                           object
Project Phase Actual Start Date                               object
Project Phase Planned End Date                                object
Project Phase Actual End Date                                 object
Project Budget Amount                                         object
Final Estimate of Actual Costs Through End of Phase Amount    object
Total Phase Actual Spending Amount                            object
DSF Number(s)                                                 object
dtype: object

In [6]:
b = []

# Create a new row per unique DSF, even if a row has more then 1
for index, row in budgets.iterrows(): 
    for dsf in re.findall('(\d+)', row['DSF Number(s)']):
        b.append([
                  "DSF%s"%(dsf),
                  row['Project Building Identifier'],
                  row['Project School Name'],
                  row['Project Phase Actual Start Date'],
                  row['Project Phase Planned End Date']
                 ])
        
budgetsClean = pd.DataFrame(data=b, columns=['dsf', 'bldg_id', 'name', 'start_date', 'planned_end_date'])

In [7]:
budgetsClean = budgetsClean.drop_duplicates(subset='dsf')

In [28]:
# Data from: https://data.cityofnewyork.us/Education/Capacity-Projects-by-schools/a94k-kjys
# As of: 03/21/2018

projects = pd.read_csv('data/Capacity_Projects__by_schools.csv')

In [29]:
projects.sample(5)

,Existing Site Identified,Proposed Leased Facility,District,Project #,School,Boro,Forecast Capacity,Design Start,Constr Start,Actual /Est.Compl,Total Est.Cost,PreviousAppropriations,FundingReq'dFY 15-19,Needed toComplete
39,No,Yes,20,DSF0000798188,PROJECT #2,NaN,640,Mar-18,Jun-19,Jun-22,28.98,0.00,25.90,3.09
41,No,No,20,DSF0000798190,PROJECT #7,NaN,456,Jun-18,Sep-19,Jun-22,34.08,0.00,2.57,31.51
29,Yes,Yes,15,DSF0000799396,P.S. 516 SUNSET PARK AVENUES ELEMENTARY SCHOOL,NaN,113,Jan-14,Jul-14,Jul-16,11.60,0.12,11.49,0.00
21,No,No,13,DSF0000843661,PROJECT #3,NaN,656,Jun-18,Sep-19,Jun-22,50.63,0.00,4.83,45.80
15,Yes,No,11,DSF0000798178,P.S. 97,NaN,548,Feb-17,Apr-18,Apr-21,93.29,0.11,85.03,8.15


In [17]:
projects = pd.DataFrame(
                        data=projects.loc[:, ['Project #', 'Forecast Capacity', 'Actual /Est.Compl']].values,
                        columns=['dsf', 'capacity', 'est_completion']
                       )

In [18]:
projects = projects.drop_duplicates(subset='dsf')

In [64]:
# Data from: https://data.cityofnewyork.us/Housing-Development/Active-Projects-Under-Construction/8586-3zfm
# As of: 02/22/2018

locations = pd.read_csv('data/Active_Projects_Under_Construction.csv')

In [20]:
locations = pd.DataFrame(
                         data=locations.loc[:, ['Building ID', 'Latitude', 'Longitude', 'BBL']].values,
                         columns=['bldg_id', 'lat', 'lng', 'bbl']
                        )

In [21]:
locations = locations.drop_duplicates(subset=['bldg_id']).dropna(subset=['bbl'])

In [24]:
budgetsClean.sample(5)

,dsf,bldg_id,name,start_date,planned_end_date
17,DSF0000798239,K653,I.S. 653 - BROOKLYN,06/07/2016,07/20/2016
43,DSF0000798189,K127,P.S. 127 - BROOKLYN,07/14/2017,02/22/2018
8,DSF0000798185,K032,P.S. 32 - BROOKLYN,07/12/2017,07/10/2021
23,DSF0000798235,M323,I.S. 323 - MANHATTAN,03/09/2015,06/15/2017
41,DSF0000822081,K667,P.S./I.S. @ 3269 ATLANTIC AVE,08/08/2017,04/04/2018


In [120]:
df = budgetsClean.merge(projects, on='dsf', how='inner').merge(locations, on='bldg_id', how='inner')

In [121]:
df

,dsf,bldg_id,name,start_date,planned_end_date,capacity,est_completion,lat,lng,bbl
0,DSF0000798178,X097,P.S. 97 - BRONX,07/31/2017,02/12/2018,548,Apr-21,40.8628,-73.8468,2.04474e+09
1,DSF0000730952,M342,THE RIVERSIDE SCHOOL - M,06/17/2016,09/01/2017,692,Sep-17,40.7725,-73.9898,1.01172e+09
2,DSF0000798185,K032,P.S. 32 - BROOKLYN,07/12/2017,07/10/2021,436,Jul-20,40.6804,-73.9919,3.00437e+09
3,DSF0000798182,K557,P.S. @ 4302 4TH AVE - BROOKLYN,11/17/2017,06/12/2018,332,Jun-22,40.6503,-74.0086,3.00728e+09
4,DSF0000798233,X014,P.S. 14 - BRONX,07/07/2016,10/09/2019,344,Sep-18,40.8358,-73.8257,2.05313e+09
5,DSF0000798175,X046,P.S. 46 - BRONX,01/03/2017,06/25/2020,500,Jun-19,40.8668,-73.8907,2.03295e+09
6,DSF0000798235,M323,I.S. 323 - MANHATTAN,03/09/2015,06/15/2017,1016,May-18,40.731,-74.0072,1.00603e+09
7,DSF0000798210,Q398,P.S. 398 - QUEENS,03/06/2017,06/03/2019,476,Jun-19,40.7525,-73.8969,4.01242e+09
8,DSF0000798199,Q125,I.S. 125 - QUEENS,12/23/2015,07/22/2018,728,Sep-17,40.7411,-73.9189,4.02284e+09
9,DSF0000798195,Q143,P.S. 143 - QUEENS,07/18/2017,03/17/2021,980,Sep-20,40.7555,-73.8552,4.01756e+09


In [126]:
df['start_date'] = pd.to_datetime(df['start_date'])
df['planned_end_date'] = pd.to_datetime(df['planned_end_date'])
df['est_completion'] = pd.to_datetime(df['est_completion'], format="%b-%y")

In [129]:
df.to_csv('export/sca-capital-projects.csv')